In [16]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime

In [10]:
REQ = requests.get('https://globefunder.com/revolut-stocks-list/') 

In [19]:
soup = BeautifulSoup(REQ.content,'lxml')
table = soup.find('table')
str_tab = str(table)
df = pd.read_html(str_tab)[0].drop(columns=['#'])
df.to_csv('Revolut_stocks_'+datetime.now().strftime('%Y-%m-%d')+'.csv')
df

,Company name,Symbol,Stock price (USD),Sector,Industry,Market
0,Agilent Technologies,A,$154.75,Health Technology,Medical Specialties,NYSE
1,Alcoa,AA,$57.11,Non-Energy Minerals,Aluminum,NYSE
2,American Airlines,AAL,$20.02,Transportation,Airlines,NASDAQ
3,Advance Auto Parts,AAP,$220.99,Retail Trade,Specialty Stores,NYSE
4,Apple,AAPL,$144.52,Electronic Technology,Telecommunications Equipment,NASDAQ
...,...,...,...,...,...,...
941,Zynga,ZNGA,$7.37,Technology Services,Internet Software/Services,NASDAQ
942,China Southern Airlines,ZNH,$30.79,Transportation,Airlines,NYSE
943,Zscaler,ZS,$298.59,Technology Services,Packaged Software,NASDAQ
944,ZTO Express,ZTO,$31.02,Transportation,Air Freight/Couriers,NYSE


In [8]:
y_df = df[:5]
y_df

,Company name,Symbol,Stock price (USD),Sector,Industry,Market
0,Agilent Technologies,A,$154.75,Health Technology,Medical Specialties,NYSE
1,Alcoa,AA,$57.11,Non-Energy Minerals,Aluminum,NYSE
2,American Airlines,AAL,$20.02,Transportation,Airlines,NASDAQ
3,Advance Auto Parts,AAP,$220.99,Retail Trade,Specialty Stores,NYSE
4,Apple,AAPL,$144.52,Electronic Technology,Telecommunications Equipment,NASDAQ


In [11]:
ld = []

for ticker in df.Symbol[:5]:
    stock = yf.Ticker(ticker)
    info=stock.info
    enviro = stock.sustainability 
    info_of_interest = ['profitMargins','fullTimeEmployees','trailingEps','lastDividendDate','dividendRate','trailingPE','marketCap','dividendYield','quickRatio','payoutRatio','shortPercentOfFloat','bookValue']
    enviromental_data_of_interest = ['environmentScore','palmOil','furLeather','catholic','animalTesting','highestControversy']
    upcoming_earnings_date = stock.calendar.T['Earnings Date'].values
    
    d={'Symbol':ticker,'nextEarningsDate':upcoming_earnings_date}
    
    #stats
    for col in info_of_interest:
        try:
            d[col] = info[col]
        except:
            d[col] = np.nan
    
    #sustainability
    for col in enviromental_data_of_interest:
        try:
            d[col] = info[col]
        except:
            d[col] = np.nan

    #last quarter earnings and revenue
    d.update(stock.quarterly_earnings.iloc[-1].rename({'Revenue':'lastQuarterRevenue','Earnings':'lastQuarterEarnings'}).to_dict())

    ld.append(d)
    
ex_df = pd.DataFrame(ld) 
    
ex_df


,Symbol,nextEarningsDate,profitMargins,fullTimeEmployees,trailingEps,lastDividendDate,dividendRate,trailingPE,marketCap,dividendYield,...,shortPercentOfFloat,bookValue,environmentScore,palmOil,furLeather,catholic,animalTesting,highestControversy,lastQuarterRevenue,lastQuarterEarnings
0,A,[2021-11-22T21:00:00.000000000],0.16119,16700,3.208,1633305600,0.78,49.311100,47887753216,0.0049,...,0.0138,16.329,NaN,NaN,NaN,NaN,NaN,NaN,1586000000,264000000
1,AA,"[2022-01-18T10:59:00.000000000, 2022-01-24T12:...",0.07292,12900,4.320,1478044800,0.40,11.469907,9268823040,0.0078,...,0.0503,20.731,NaN,NaN,NaN,NaN,NaN,NaN,3109000000,337000000
2,AAL,"[2022-01-26T13:30:00.000000000, 2022-01-31T13:...",-0.13229,119800,-5.184,1580774400,NaN,NaN,12555295744,NaN,...,0.1564,-11.486,NaN,NaN,NaN,NaN,NaN,NaN,8969000000,169000000
3,AAP,"[2021-11-08T13:30:00.000000000, 2021-11-12T13:...",0.05732,40000,9.283,1631750400,4.00,25.052246,14651442176,0.0174,...,0.0764,52.440,NaN,NaN,NaN,NaN,NaN,NaN,2649415000,178696000
4,AAPL,[2021-10-28T20:00:00.000000000],0.25004,147000,5.108,1628208000,0.88,29.232578,2468289445888,0.0059,...,0.0061,3.882,NaN,NaN,NaN,NaN,NaN,NaN,81434000000,21744000000


In [12]:
merged = pd.merge(y_df,ex_df,on='Symbol')
merged

,Company name,Symbol,Stock price (USD),Sector,Industry,Market,nextEarningsDate,profitMargins,fullTimeEmployees,trailingEps,...,shortPercentOfFloat,bookValue,environmentScore,palmOil,furLeather,catholic,animalTesting,highestControversy,lastQuarterRevenue,lastQuarterEarnings
0,Agilent Technologies,A,$154.75,Health Technology,Medical Specialties,NYSE,[2021-11-22T21:00:00.000000000],0.16119,16700,3.208,...,0.0138,16.329,NaN,NaN,NaN,NaN,NaN,NaN,1586000000,264000000
1,Alcoa,AA,$57.11,Non-Energy Minerals,Aluminum,NYSE,"[2022-01-18T10:59:00.000000000, 2022-01-24T12:...",0.07292,12900,4.320,...,0.0503,20.731,NaN,NaN,NaN,NaN,NaN,NaN,3109000000,337000000
2,American Airlines,AAL,$20.02,Transportation,Airlines,NASDAQ,"[2022-01-26T13:30:00.000000000, 2022-01-31T13:...",-0.13229,119800,-5.184,...,0.1564,-11.486,NaN,NaN,NaN,NaN,NaN,NaN,8969000000,169000000
3,Advance Auto Parts,AAP,$220.99,Retail Trade,Specialty Stores,NYSE,"[2021-11-08T13:30:00.000000000, 2021-11-12T13:...",0.05732,40000,9.283,...,0.0764,52.440,NaN,NaN,NaN,NaN,NaN,NaN,2649415000,178696000
4,Apple,AAPL,$144.52,Electronic Technology,Telecommunications Equipment,NASDAQ,[2021-10-28T20:00:00.000000000],0.25004,147000,5.108,...,0.0061,3.882,NaN,NaN,NaN,NaN,NaN,NaN,81434000000,21744000000
